# Create a QPN matched sub sample from PPMI
    1. Using propensity score matching, including age, sex as covarites to match;
    2. tbd
    
    Visit encoding:

      1. Normal visits:
         'Screening->0',
         'Baseline->1',
         'Month 12->5',
         'Month 24->7',
         'Month 36->9',
         'Month 48->11',

      2. Abnornal visits:
         'Premature Withdrawal->30',
         'Symptomatic Therapy->21',
         'Unscheduled Visit 01->90',
         'Unscheduled Visit 02->91'

In [1]:
from pathlib import Path
import pandas as pd

#PATH
tmp_dir  = Path("/scratch/0tmp") # For privacy, all the PPMI data are stored here locally.
main_dir = Path("/scratch/mr_proc/metadata") # The main GitRepo meta data dir

qpn_file  = main_dir / "qpn"  / "participants.csv" # qpn participants table.
ppmi_file = main_dir / "ppmi" / "participants.csv" # PPMI participants table.
ppmi_all_file = main_dir / "ppmi" / "PPMI_3T_sdMRI_3_07_2022.csv" # The imaging downloading table from livingPark. 

In [2]:
# Known T1 sequence descriptions,copied from heuristics.
T1W_SERIES = [
    'MPRAGE 2 ADNI',
    'MPRAGE ADNI',
    'MPRAGE GRAPPA 2',
    'MPRAGE GRAPPA2',
    'MPRAGE GRAPPA2(adni)',
    'MPRAGE w/ GRAPPA',
    'MPRAGE_GRAPPA',
    'MPRAGE_GRAPPA_ADNI',
    'MPRAGE GRAPPA',
    'SAG T1 3D MPRAGE',
    'sag mprage',
    'MPRAGEadni',
    'MPRAGE GRAPPA_ND',
    '3D SAG',
    'MPRAGE T1 SAG',
    'MPRAGE SAG',
    'SAG T1 3DMPRAGE',
    'SAG T1 MPRAGE',
    'SAG 3D T1',
    'SAG MPRAGE GRAPPA2-NEW2016',
    'SAG MPRAGE GRAPPA_ND',
    'Sag MPRAGE GRAPPA',
    'AXIAL T1 3D MPRAGE',
    'SAG MPRAGE GRAPPA',
    'sT1W_3D_FFE',
    'sT1W_3D_ISO',
    'sT1W_3D_TFE',
    'sag 3D FSPGR BRAVO straight',
    'SAG T1 3D FSPGR',
    'SAG FSPGR 3D '
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG T1 3D FSPGR 3RD REPEAT',
    'SAG FSPGR BRAVO',
    'SAG SPGR 3D',
    'SAG 3D SPGR',
    'FSPGR 3D SAG',
    'SAG FSPGR 3D',
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG FSPGR 3D ',
    't1_mpr_ns_sag_p2_iso',
    'T1',
    'T1 Repeat',
    'AX T1',
    'axial spgr',
    'T1W_3D_FFE AX',
    # added by Vincent
    'AX T1 SE C+',
    '3D SAG T1 MPRAGE',
    '3D SAG T1 MPRAGE_ND',
    '3D T1',
    '3D T1 MPRAGE',
    '3D T1-weighted',
    'Accelerated Sag IR-FSPGR',
    'MPRAGE',
    'MPRAGE - Sag',
    'MPRAGE Phantom GRAPPA2',
    'MPRAGE w/ GRAPPA 2',
    'PPMI_MPRAGE_GRAPPA2',
    'SAG 3D T1 FSPGR',
    'SAG FSPGR 3D VOLUMETRIC T1',
    'Sag MPRAGE GRAPPA_ND',
    'T1-weighted, 3D VOLUMETRIC',
    'tra_T1_MPRAGE',
    '3D T1-weighted_ND', ## added from livingpark
    '3D T1 _weighted',
    'Sagittal 3D Accelerated MPRAGE',
    'T1 REPEAT',
    'MPRAGE Repeat',
    'SAG_3D_MPRAGE'
]
def check_T1(s_):
    res=s_.copy()
    for i in range(len(s_)):
        if s_[i] in T1W_SERIES:
            res[i]=True
        else:
            res[i]=False
    return res

In [3]:
participant_col_sel = ['participant_id', 'age', 'sex', 'group']
metadata_col_sel    = ['Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality', 'Description']

# harmonizing group label
qpn_group2ppmi_group={'Parkinson':'PD', 'Parkison':'PD', 'Parkinson ':'PD', 'Control ':'Control', 'Control':'Control', 'PD':'PD'};

# Read ppmi subjects
ppmi_df  = pd.read_csv(ppmi_file, sep=',', )
ppmi_df=ppmi_df.sort_values(by="participant_id").reset_index(drop=True)
print('PPMI subject tab info: ', ppmi_df.shape)

# Read qpn subjects
qpn_df  = pd.read_csv(qpn_file, sep=',')
qpn_df=qpn_df.sort_values(by="participant_id").reset_index(drop=True)
qpn_df.loc[:,'group']=qpn_df.loc[:,'group'].apply(lambda x:qpn_group2ppmi_group[x])
print('QPN subject tab info: ', qpn_df.shape)

# Read ppmi all info
ppmi_all_df = pd.read_csv(ppmi_all_file, sep=',')
ppmi_all_df = ppmi_all_df.loc[:, metadata_col_sel]
# filtering group (PD and Control) and modality (T1)
ppmi_all_df = ppmi_all_df[(ppmi_all_df['Modality']=='MRI') & (check_T1(ppmi_all_df['Description']))].copy()
ppmi_all_df = ppmi_all_df.drop(columns=['Modality', 'Description']).copy()
ppmi_all_df = ppmi_all_df.sort_values(by="Subject").reset_index(drop=True)
ppmi_all_df.columns=['Image ID', 'participant_id', 'group', 'sex', 'age', 'visit'];
ppmi_all_df = ppmi_all_df[(ppmi_all_df['group']=='PD') | (ppmi_all_df['group']=='Control')].copy()
# reorder columns
ppmi_all_df=ppmi_all_df.loc[:, ['Image ID', 'participant_id', 'age', 'sex', 'group', 'visit']]
print('PPMI image tab info: ', ppmi_all_df.shape)

PPMI subject tab info:  (585, 4)
QPN subject tab info:  (296, 4)
PPMI image tab info:  (1027, 6)
